In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import random

MIN_LIST_LEN = 8
MAX_LIST_LEN = 8
MAX_STEPS = 210

SUCCESS_REWARD = 2.0
STEP_REWARD = -0.2
COMPARISON_ENTROPY_MULTIPLIER = -0.05
SWAP_REWARD = 1.0
INVALID_ACTION_REWARD = -10.0

EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 1000
GAMMA = 0.9
NUM_EPISODES = 100000
EPISODES_SAVE = 1000
OUTPUT_DIR = 'datasets/rl_sort_transformer_easy/list8_transformer4_192_gamma09_step210_v2'

# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    'less': 2,
    'equal': 3,
    'more': 4,
    '0': 5,
    '1': 6,
    '2': 7,
    '3': 8,
    '4': 9,
    '5': 10,
    '6': 11,
    '7': 12,
    '8': 13,
    '9': 14,
    '10': 15,
    '11': 16,
    '12': 17,
    '13': 18,
    '14': 19,
    '15': 20,
    'len1': 21,
    'len2': 22,
    'len3': 23,
    'len4': 24,
    'len5': 25,
    'len6': 26,
    'len7': 27,
    'len8': 28,
    'len9': 29,
    'len10': 30,
    'len11': 31,
    'len12': 32,
    'len13': 33,
    'len14': 34,
    'len15': 35,
    'len16': 36,
}
inv_vocab = {v: k for k, v in vocab.items()}

def compute_entropy(N, alpha=1):
    K = 2**N
    values = np.arange(K)
    unnormalized_probs = np.exp(-alpha * values)
    Z = unnormalized_probs.sum()
    probs = unnormalized_probs / Z
    return values, -np.log2(probs)

_, int_entropy = compute_entropy(4)

def get_entropy_of_integer(x):
    x = min(15, abs(x))
    return int_entropy[x]

def compute_min_delta_entropy(comparisons):
    # Initialize the result list to store minDelta values
    min_delta = None

    # Iterate through each pair in the comparisons list
    i = len(comparisons) - 1
    xi, yi = comparisons[i]
    if i == 0:
        # For i = 0, use the first case directly
        min_delta = (xi, min(yi, yi - xi), 0)
    else:
        # For i > 0, compute all possible options and select the minimal one
        options = []

        # Simple Entropy
        simple_entropy = (xi, min(yi, yi - xi), 0)
        options.append(simple_entropy)

        # First Delta Entropy
        xi_prev, yi_prev = comparisons[i - 1]
        first_delta_entropy = (xi - xi_prev, yi - yi_prev, 0)
        options.append(first_delta_entropy)

        # Second Delta Entropy (only valid for i > 1)
        if i > 1:
            xi_prev2, yi_prev2 = comparisons[i - 2]
            second_delta_entropy = (
                (xi - xi_prev) - (xi_prev - xi_prev2),
                (yi - yi_prev) - (yi_prev - yi_prev2),
                0,
            )
            options.append(second_delta_entropy)

        # Arbitrary Position Entropy (only valid for i > 1)
        for j in range(i):
            xj, yj = comparisons[j]
            arbitrary_position_entropy = (
                xi - xj,
                yi - yj,
                min(j, i - j),
            )
            options.append(arbitrary_position_entropy)

        # Find the option with the minimal sum
        min_delta = min(options, key=lambda t: sum([get_entropy_of_integer(x) for x in t]))

    return sum([get_entropy_of_integer(x) for x in min_delta])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN, MAX_LIST_LEN)
        self.list = [random.randint(1, 100) for _ in range(self.length)]
        while self.list == sorted(self.list):
            self.list = [random.randint(1, 100) for _ in range(self.length)]
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_list_len(self):
        return len(self.list)

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = STEP_REWARD
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = STEP_REWARD * 2
            else:
                response_token = vocab['more']
                reward = STEP_REWARD
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = INVALID_ACTION_REWARD
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = SUCCESS_REWARD
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = SWAP_REWARD
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -SWAP_REWARD
            else:
                reward = STEP_REWARD
            self.indices = None
        else:
            reward = INVALID_ACTION_REWARD
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy()


Using device: cuda


In [2]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=512):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=192, nhead=8, num_layers=4):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [ ]:
# Training Loop
def train(verbose=False):
    # Removed torch.autograd.set_detect_anomaly(True)
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduced learning rate
    # Optionally, load a checkpoint
    checkpoint_path = os.path.join(OUTPUT_DIR, "ckpt_52000_0.6910_169.16.pth")
    load_checkpoint(checkpoint_path, model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    num_successes = 0
    total_steps = 0
    
    for episode in range(NUM_EPISODES):
        t1 = time.time()
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        comparisons = []
        
        state = 'expect_action'
        done = False
        success = False

        while not done and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
                print(comparisons)
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
                output = model(input_seq)  # (seq_len, batch_size, vocab_size)
                # Get logits for the last token
                logits = output[-1, 0, :]  # (vocab_size)

                # Check for NaNs in logits
                if torch.isnan(logits).any():
                    print(f"Episode {episode}, NaNs in logits before masking.")
                    break

                # Get valid tokens based on state
                def get_valid_tokens(state):
                    action_tokens = [vocab['Comparison'], vocab['Swap']]
                    index_tokens = [vocab[str(i)] for i in range(env.length)]
                    if state == 'expect_action':
                        return action_tokens
                    elif state == 'expect_index1':
                        return index_tokens[:-1]
                    elif state == 'expect_index2':
                        return [x for x in index_tokens if x > input_tokens[-1]]
                    else:
                        # Handle unexpected states by defaulting to expect_action
                        return action_tokens

                valid_token_ids = get_valid_tokens(state)

                # Ensure valid_token_ids are within the vocab range
                if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                    print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                    break

                # Mask invalid tokens
                mask_value = -1e9  # Use a large negative value instead of -inf
                mask = torch.full_like(logits, mask_value).to(device)
                mask[valid_token_ids] = 0
                masked_logits = logits + mask

                # Sample action. Have some chance to randomly pick a valid action.
                eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
                if random.random() < eps_threshold:
                    masked_logits = masked_logits / 4

                # Check for NaNs in masked_logits
                if torch.isnan(masked_logits).any():
                    print(f"Episode {episode}, NaNs in masked_logits after masking.")
                    break

                # Compute probabilities
                probs = F.softmax(masked_logits, dim=0)

                # Check for NaNs in probs
                if torch.isnan(probs).any():
                    print(f"Episode {episode}, NaNs in probs after softmax.")
                    break

                try:
                    m = torch.distributions.Categorical(probs)
                    action_token = m.sample()
                    log_prob = m.log_prob(action_token)
                except ValueError as e:
                    print(f"Episode {episode}, error in sampling action: {e}")
                    break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        reward = INVALID_ACTION_REWARD
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    if done and reward == SUCCESS_REWARD:
                        success = True
                        #print(decode(input_tokens, inv_vocab))
                    if verbose:
                        print("Reward:", reward)
                    state = 'expect_action'
                else:
                    reward = INVALID_ACTION_REWARD
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                comparisons.append((int(inv_vocab[index1_token.item()]), 
                                    int(inv_vocab[index2_token.item()])))
                response_token, reward, done, current_list = env.step(action_tokens)
                if done and reward == SUCCESS_REWARD:
                    success = True
                    #print(decode(input_tokens, inv_vocab))
                else:
                    reward += COMPARISON_ENTROPY_MULTIPLIER * compute_min_delta_entropy(comparisons)
                if verbose:
                    print("Reward:", reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                reward = INVALID_ACTION_REWARD
                done = True

            rewards.append(reward)
        #
        if success: 
            num_successes += 1
        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / episode_cnt
            success_rate = num_successes / episode_cnt
            avg_steps = total_steps / episode_cnt
            episode_cnt = 0
            total_reward = 0.0
            num_successes = 0
            total_steps = 0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{success_rate:.4f}_{avg_steps:.2f}.pth")
        #
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"

        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns and loss within autocast
        with torch.autocast(device_type=device.type, dtype=torch.bfloat16):
            # Compute returns
            returns = []
            R = 0
            gamma = GAMMA
            for r in rewards[::-1]:
                R = r + gamma * R
                returns.insert(0, R)
            returns = torch.tensor(returns).to(device)

            # Check for NaNs in returns
            if torch.isnan(returns).any():
                print(f"Episode {episode}, NaNs in returns.")
                continue

            # Compute loss
            loss = 0
            for log_prob, R in zip(log_probs, returns):
                loss -= log_prob * R

            # Check for NaNs in loss
            if torch.isnan(loss):
                print(f"Episode {episode}, NaN in loss.")
                continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        total_steps += len(rewards)
        t2 = time.time()
        if episode % 1 == 0:
            print(f"Episode {episode}, loss:{loss.item():.4f}, {'succeed' if success else 'fail'}, steps:{len(rewards)}, total reward:{sum(rewards):.4f}, {t2-t1} sec")

if __name__ == "__main__":
    train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


No checkpoint found at datasets/rl_sort_transformer_easy/list8_transformer4_192_gamma09_step210_v2/ckpt_52000_0.6910_169.16.pth, starting from scratch.
Episode 0, loss:-68.0615, fail, steps:12, total reward:-11.6913, 0.361278772354126 sec
Episode 1, loss:-14.7772, fail, steps:1, total reward:-10.0000, 0.0053904056549072266 sec
Episode 2, loss:-72.0592, fail, steps:8, total reward:-12.2477, 0.020350933074951172 sec
Episode 3, loss:-181.0044, fail, steps:159, total reward:-20.3830, 0.4952394962310791 sec
Episode 4, loss:-156.1559, fail, steps:158, total reward:-21.3937, 0.4323863983154297 sec
Episode 5, loss:-121.6667, fail, steps:158, total reward:-18.3224, 0.4314444065093994 sec
Episode 6, loss:-157.3369, fail, steps:158, total reward:-23.2601, 0.4921417236328125 sec
Episode 7, loss:-106.3106, fail, steps:158, total reward:-19.3838, 0.4371950626373291 sec
Episode 8, loss:-122.1537, fail, steps:78, total reward:-20.0783, 0.2270948886871338 sec
Episode 9, loss:-84.5488, fail, steps:158, 

Episode 89, loss:-70.4431, fail, steps:160, total reward:-10.3001, 0.4361541271209717 sec
Episode 90, loss:-116.6133, fail, steps:158, total reward:-18.1674, 0.4342975616455078 sec
Episode 91, loss:-131.2911, fail, steps:158, total reward:-17.8822, 0.43535399436950684 sec
Episode 92, loss:-103.5456, fail, steps:159, total reward:-14.7445, 0.44062185287475586 sec
Episode 93, loss:-119.9216, fail, steps:159, total reward:-13.5895, 0.4430429935455322 sec
Episode 94, loss:-97.5164, fail, steps:158, total reward:-15.3117, 0.44444799423217773 sec
Episode 95, loss:-97.9567, fail, steps:159, total reward:-14.6723, 0.44460630416870117 sec
Episode 96, loss:-146.0208, fail, steps:158, total reward:-19.1150, 0.5019218921661377 sec
Episode 97, loss:-110.6428, fail, steps:158, total reward:-15.9609, 0.44249820709228516 sec
Episode 98, loss:-91.6464, fail, steps:159, total reward:-13.0845, 0.4474372863769531 sec
Episode 99, loss:-119.9190, fail, steps:158, total reward:-18.5773, 0.4437685012817383 se

Episode 179, loss:-122.2065, fail, steps:160, total reward:-12.9699, 0.4394955635070801 sec
Episode 180, loss:-137.0694, fail, steps:159, total reward:-15.1666, 0.43956851959228516 sec
Episode 181, loss:-108.7911, fail, steps:160, total reward:-11.3459, 0.44884777069091797 sec
Episode 182, loss:-60.2971, fail, steps:160, total reward:-7.2181, 0.4455752372741699 sec
Episode 183, loss:-68.8557, fail, steps:160, total reward:-9.1459, 0.4458889961242676 sec
Episode 184, loss:-57.0074, fail, steps:160, total reward:-6.7131, 0.44843339920043945 sec
Episode 185, loss:-112.2681, fail, steps:159, total reward:-15.0330, 0.44401097297668457 sec
Episode 186, loss:-96.9544, fail, steps:160, total reward:-10.7952, 0.4511110782623291 sec
Episode 187, loss:-68.9890, fail, steps:160, total reward:-9.4246, 0.45188260078430176 sec
Episode 188, loss:-65.5376, fail, steps:161, total reward:-6.4860, 0.45093369483947754 sec
Episode 189, loss:-148.4350, fail, steps:155, total reward:-18.5483, 0.43459606170654

Episode 269, loss:-146.7460, fail, steps:159, total reward:-14.7895, 0.4132668972015381 sec
Episode 270, loss:-118.5549, succeed, steps:148, total reward:-12.3081, 0.4026782512664795 sec
Episode 271, loss:-97.2145, fail, steps:160, total reward:-8.9394, 0.44368672370910645 sec
Episode 272, loss:-133.9763, fail, steps:160, total reward:-13.0214, 0.4430713653564453 sec
Episode 273, loss:-120.4008, fail, steps:159, total reward:-12.7436, 0.44463396072387695 sec
Episode 274, loss:-109.5324, fail, steps:160, total reward:-10.3886, 0.4472992420196533 sec
Episode 275, loss:-135.4952, fail, steps:29, total reward:-12.8475, 0.09536147117614746 sec
Episode 276, loss:-116.6146, fail, steps:160, total reward:-11.5886, 0.41971874237060547 sec
Episode 277, loss:-64.0904, fail, steps:161, total reward:-6.1459, 0.43619275093078613 sec
Episode 278, loss:-90.0499, fail, steps:161, total reward:-8.5058, 0.44405484199523926 sec
Episode 279, loss:-129.2678, fail, steps:45, total reward:-10.6937, 0.19273614

...

Episode 19185, loss:-3.7600, succeed, steps:134, total reward:2.5943, 0.3744547367095947 sec
Episode 19186, loss:-0.6523, succeed, steps:101, total reward:-0.5845, 0.28061580657958984 sec
Episode 19187, loss:-18.0604, fail, steps:159, total reward:-10.1468, 0.427170991897583 sec
Episode 19188, loss:-2.5161, succeed, steps:88, total reward:0.0224, 0.24891185760498047 sec
Episode 19189, loss:-7.0779, fail, steps:163, total reward:3.4625, 0.4359700679779053 sec
Episode 19190, loss:-9.2740, fail, steps:161, total reward:-2.2696, 0.44162702560424805 sec
Episode 19191, loss:5.4813, succeed, steps:93, total reward:1.9625, 0.2719078063964844 sec
Episode 19192, loss:-2.0819, succeed, steps:146, total reward:-0.7832, 0.3958864212036133 sec
Episode 19193, loss:-19.4967, fail, steps:160, total reward:-7.1459, 0.4411797523498535 sec
Episode 19194, loss:1.1681, succeed, steps:38, total reward:0.9933, 0.1177983283996582 sec
Episode 19195, loss:-2.7353, fail, steps:162, total reward:1.7510, 0.41736936

Episode 19274, loss:3.3289, succeed, steps:78, total reward:3.8194, 0.21809625625610352 sec
Episode 19275, loss:-0.4737, succeed, steps:64, total reward:3.8821, 0.17263293266296387 sec
Episode 19276, loss:-3.1866, succeed, steps:128, total reward:3.1764, 0.3320941925048828 sec
Episode 19277, loss:6.2980, succeed, steps:106, total reward:8.5804, 0.29424333572387695 sec
Episode 19278, loss:-2.4882, succeed, steps:118, total reward:-3.3079, 0.3177924156188965 sec
Episode 19279, loss:-0.4250, succeed, steps:82, total reward:1.2931, 0.2254018783569336 sec
Episode 19280, loss:-1.8123, fail, steps:161, total reward:-3.1253, 0.42627382278442383 sec
Episode 19281, loss:0.7001, succeed, steps:49, total reward:2.0791, 0.1428995132446289 sec
Episode 19282, loss:-0.5573, succeed, steps:67, total reward:-0.4378, 0.1756424903869629 sec
Episode 19283, loss:-7.9448, fail, steps:161, total reward:-3.8761, 0.4177875518798828 sec
Episode 19284, loss:-11.5999, fail, steps:161, total reward:-1.6925, 0.46732

Episode 19363, loss:-14.5121, fail, steps:161, total reward:-6.0017, 0.4202442169189453 sec
Episode 19364, loss:-15.2886, fail, steps:161, total reward:-5.2803, 0.43634653091430664 sec
Episode 19365, loss:-6.6238, succeed, steps:97, total reward:-4.2960, 0.27544236183166504 sec
Episode 19366, loss:-18.4676, fail, steps:159, total reward:-12.7239, 0.4262564182281494 sec
Episode 19367, loss:-4.0973, succeed, steps:108, total reward:3.4933, 0.3042469024658203 sec
Episode 19368, loss:0.6813, succeed, steps:34, total reward:4.0082, 0.1027071475982666 sec
Episode 19369, loss:-0.9655, succeed, steps:114, total reward:2.0849, 0.29517674446105957 sec
Episode 19370, loss:-3.3446, succeed, steps:88, total reward:-3.4703, 0.23954153060913086 sec
Episode 19371, loss:-6.8874, fail, steps:162, total reward:-1.1975, 0.4259519577026367 sec
Episode 19372, loss:0.6069, succeed, steps:108, total reward:-0.2552, 0.3015899658203125 sec
Episode 19373, loss:1.1030, succeed, steps:106, total reward:4.8483, 0.2

Episode 19453, loss:-4.2349, fail, steps:159, total reward:-11.8746, 0.42429542541503906 sec
Episode 19454, loss:2.0702, fail, steps:163, total reward:5.4109, 0.4449641704559326 sec
Episode 19455, loss:-4.0371, succeed, steps:78, total reward:-2.7069, 0.22691106796264648 sec
Episode 19456, loss:-2.2959, fail, steps:161, total reward:-4.1262, 0.48272132873535156 sec
Episode 19457, loss:1.0287, succeed, steps:81, total reward:3.5923, 0.23179292678833008 sec
Episode 19458, loss:-6.4880, fail, steps:160, total reward:-10.3632, 0.45101332664489746 sec
Episode 19459, loss:-1.4635, fail, steps:161, total reward:-4.9918, 0.4397563934326172 sec
Episode 19460, loss:-8.2262, fail, steps:159, total reward:-11.2910, 0.44072532653808594 sec
Episode 19461, loss:-6.0084, fail, steps:160, total reward:-10.2353, 0.4437987804412842 sec
Episode 19462, loss:-0.0592, fail, steps:161, total reward:-0.8269, 0.44729113578796387 sec
Episode 19463, loss:1.3597, succeed, steps:89, total reward:2.1995, 0.261684417

Episode 19542, loss:-8.1661, fail, steps:159, total reward:-11.6616, 0.4372878074645996 sec
Episode 19543, loss:0.4317, succeed, steps:106, total reward:1.9097, 0.3035552501678467 sec
Episode 19544, loss:0.9623, succeed, steps:49, total reward:1.8070, 0.13893508911132812 sec
Episode 19545, loss:5.5081, succeed, steps:98, total reward:6.8903, 0.255321741104126 sec
Episode 19546, loss:-9.9398, fail, steps:161, total reward:-5.4246, 0.48016810417175293 sec
Episode 19547, loss:0.5689, fail, steps:161, total reward:-4.6311, 0.4375951290130615 sec
Episode 19548, loss:0.8210, succeed, steps:102, total reward:4.1476, 0.28929972648620605 sec
Episode 19549, loss:-9.7333, fail, steps:161, total reward:-2.6826, 0.43776965141296387 sec
Episode 19550, loss:-17.0065, fail, steps:159, total reward:-16.0624, 0.43877172470092773 sec
Episode 19551, loss:-4.3721, succeed, steps:141, total reward:-4.8585, 0.3923325538635254 sec
Episode 19552, loss:-2.9765, fail, steps:161, total reward:-5.2082, 0.445494413

In [ ]:
# Testing code

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def compute_entropy(N, alpha=1):
    K = 2**N
    values = np.arange(K)
    unnormalized_probs = np.exp(-alpha * values)
    Z = unnormalized_probs.sum()
    probs = unnormalized_probs / Z
    return values, -np.log2(probs)

# Parameters
N = 4  # Number of bits
alpha = 1  # Decay rate

# Compute probabilities
values, probs = compute_entropy(N, alpha)

# Plotting
plt.figure(figsize=(10, 6))
plt.bar(values, probs, width=0.8, alpha=0.7, edgecolor="black")
plt.title(f"Exponential Decay Probabilities (N={N}, alpha={alpha})")
plt.xlabel("Integer Value")
plt.ylabel("Probability")
plt.grid(axis="y", linestyle="--", alpha=0.7)
plt.show()

In [ ]:
def compute_min_delta(comparisons):
    # Initialize the result list to store minDelta values
    min_delta = []

    # Iterate through each pair in the comparisons list
    for i, (xi, yi) in enumerate(comparisons):
        if i == 0:
            # For i = 0, use the first case directly
            min_delta.append((xi, min(yi, yi - xi), 0))
        else:
            # For i > 0, compute all possible options and select the minimal one
            options = []

            # Simple Entropy
            simple_entropy = (xi, min(yi, yi - xi), 0)
            options.append(simple_entropy)

            # First Delta Entropy
            xi_prev, yi_prev = comparisons[i - 1]
            first_delta_entropy = (xi - xi_prev, yi - yi_prev, 0)
            options.append(first_delta_entropy)

            # Second Delta Entropy (only valid for i > 1)
            if i > 1:
                xi_prev2, yi_prev2 = comparisons[i - 2]
                second_delta_entropy = (
                    (xi - xi_prev) - (xi_prev - xi_prev2),
                    (yi - yi_prev) - (yi_prev - yi_prev2),
                    0,
                )
                options.append(second_delta_entropy)

            # Arbitrary Position Entropy (only valid for i > 1)
            for j in range(i):
                xj, yj = comparisons[j]
                arbitrary_position_entropy = (
                    xi - xj,
                    yi - yj,
                    min(j, i - j),
                )
                options.append(arbitrary_position_entropy)

            # Find the option with the minimal sum
            min_delta.append(min(options, key=lambda t: sum([abs(x) for x in t])))

    return min_delta


# Example Usage
comparisons = [(1, 5), (2, 7), (4, 10), (8, 15)]
result = compute_min_delta(comparisons)

# Output the results
for i, delta in enumerate(result):
    print(f"minDelta for comparison {i}: {delta}")

def compute_min_delta_entropy(comparisons):
    # Initialize the result list to store minDelta values
    min_delta = None

    # Iterate through each pair in the comparisons list
    i = len(comparisons) - 1
    xi, yi = comparisons[i]
    if i == 0:
        # For i = 0, use the first case directly
        min_delta = (xi, min(yi, yi - xi), 0)
    else:
        # For i > 0, compute all possible options and select the minimal one
        options = []

        # Simple Entropy
        simple_entropy = (xi, min(yi, yi - xi), 0)
        options.append(simple_entropy)

        # First Delta Entropy
        xi_prev, yi_prev = comparisons[i - 1]
        first_delta_entropy = (xi - xi_prev, yi - yi_prev, 0)
        options.append(first_delta_entropy)

        # Second Delta Entropy (only valid for i > 1)
        if i > 1:
            xi_prev2, yi_prev2 = comparisons[i - 2]
            second_delta_entropy = (
                (xi - xi_prev) - (xi_prev - xi_prev2),
                (yi - yi_prev) - (yi_prev - yi_prev2),
                0,
            )
            options.append(second_delta_entropy)

        # Arbitrary Position Entropy (only valid for i > 1)
        for j in range(i):
            xj, yj = comparisons[j]
            arbitrary_position_entropy = (
                xi - xj,
                yi - yj,
                min(j, i - j),
            )
            options.append(arbitrary_position_entropy)

        # Find the option with the minimal sum
        min_delta = min(options, key=lambda t: sum([get_entropy_of_integer(x) for x in t]))

    return sum([get_entropy_of_integer(x) for x in min_delta])

compute_min_delta_entropy(comparisons)

In [ ]:

def test_case_1():
    # Test with a single comparison
    comparisons = [(3, 8)]
    expected_result = [(3, 5, 0)]
    assert compute_min_delta(comparisons) == expected_result

def test_case_2():
    # Test with multiple comparisons
    comparisons = [(1, 5), (2, 7), (4, 10), (8, 15)]
    expected_result = [
        (1, 4, 0),  # Simple Entropy for the first pair
        (1, 2, 0),  # Minimal sum using First Delta Entropy
        (1, 1, 0),  # Minimal sum using First Delta Entropy
        (2, 2, 0),  # Minimal sum using First Delta Entropy
    ]
    assert compute_min_delta(comparisons) == expected_result

def test_case_3():
    # Test with comparisons where the minimal sum comes from Arbitrary Position Entropy
    comparisons = [(1, 10), (3, 12), (5, 14), (1, 11)]
    expected_result = [
        (1, 9, 0),  # Simple Entropy for the first pair
        (2, 2, 0),  # Minimal sum using First Delta Entropy
        (0, 0, 0),  # Minimal sum using First Delta Entropy
        (0, 1, 0),  # Minimal sum using Arbitrary Position Entropy
    ]
    assert compute_min_delta(comparisons) == expected_result

test_case_1()
test_case_2()
test_case_3()
print("All cases succeeded.")